<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
pip install mplfinance


In [43]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [44]:
# Get the stock data
#df = quandl.get("WIKI/FB")
#df = pd.read_csv (r'dataset/FB.csv')
# Take a look at data
#print(df.head())

from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
df.head()

df2 = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
#print(df2.head())
# Get the Adjusted Close
df = df[['Adj Close']]
# Take a look at new data
#print(df.head())

Saving FB.csv to FB (2).csv


In [45]:
# A variable for predicting 'forecast_out' days out in future
forecast_out = 1

In [46]:
# Create another column (dependent variable) shifted 'forecast_out' units up
df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
#print(df.tail())

In [47]:
# Create the independent dataset
# Convert the dataframe to numpy array
X = np.array(df.drop(['Prediction'],1))
# Remove the last 'forecast_out' rows
X = X[:-forecast_out]
#print(X)

In [48]:
# Create the dependent dataset
# Convert the dataframe to numpy array (All of the values including the NaNs)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'forecast_out' rows
y = y[:-forecast_out]
#print(y)

In [49]:
z = np.array(df2)
z = z[:-forecast_out]
print(z)

[['2012-05-21' 36.529999 36.66 ... 34.029999 34.029999 168192700]
 ['2012-05-22' 32.610001000000004 33.59 ... 31.0 31.0 101786600]
 ['2012-05-23' 31.370001000000002 32.5 ... 32.0 32.0 73600000]
 ...
 ['2021-02-19' 269.859985 270.269989 ... 261.559998 261.559998 25549900]
 ['2021-02-22' 257.950012 263.070007 ... 260.329987 260.329987 17382300]
 ['2021-02-23' 259.5 267.459991 ... 265.859985 265.859985 21905400]]


In [50]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# train_set = x_train + y_train
# test_set = x_test + y_test

In [51]:
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [52]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [53]:
# Create and train the Linear Regression model
lr = LinearRegression()
# Train model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [70]:
# Find a different 'scoring' metric as accuracy is for classification whereas this is regression
scores = cross_val_score(svr_rbf, X, y, cv=cv, n_jobs=-1)
print(scores)

[-5.56590335e-02 -3.93263527e-02 -4.21398258e-02 -3.46726831e-04
 -4.07945614e-02 -4.16294534e-02 -4.09477431e-02 -2.47486403e-02
 -2.44805773e-01 -1.44809485e-04]


In [55]:
#Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
#print(x_forecast)

In [56]:
# Print the SVR prediction for the next forecast_out days
svm_prediction = svr_rbf.predict(x_forecast)
print(f'The next {forecast_out} days by the SVR model: \n{svm_prediction}')

# Print the Linear Regression prediction for the next forecast_out days
lr_prediction = lr.predict(x_forecast)
print(f'The next {forecast_out} days by the LR model: \n{lr_prediction}')

The next 1 days by the SVR model: 
[260.62107019]
The next 1 days by the LR model: 
[264.35033855]


In [57]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# Best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svr confidence: ", svm_confidence)

# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# It provides a measure of how well observed outcomes are replicated by the model, based on the proportion of total variation of outcomes explained by the model
# Best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

svr confidence:  0.9973987334525288
lr confidence:  0.9980078844279239


In [58]:
## Creating predicted values for both models
y_pred_svr = svr_rbf.predict(x_test)
y_pred_lr = lr.predict(x_test)

In [59]:
ann_relu = Sequential()
ann_relu.add(Dense(200, input_dim=1, activation='relu'))
ann_relu.add(Dense(200, input_dim=200, activation='relu'))
ann_relu.add(Dense(200, input_dim=200, activation='relu'))
ann_relu.add(Dense(1, activation='linear'))

In [60]:
keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
ann_relu.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [61]:
ann_relu.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 81,001
Trainable params: 81,001
Non-trainable params: 0
_________________________________________________________________


In [62]:
history = ann_relu.fit(x_train, y_train, epochs=200, batch_size=32,validation_split=0.15,verbose=1)

Epoch 1/200
47/47 [==============================] - 1s 6ms/step - loss: 2862.2940 - mean_absolute_percentage_error: 22.3108 - val_loss: 200.0624 - val_mean_absolute_percentage_error: 9.9957
Epoch 2/200
47/47 [==============================] - 0s 3ms/step - loss: 90.9322 - mean_absolute_percentage_error: 5.6444 - val_loss: 172.7009 - val_mean_absolute_percentage_error: 8.9894
Epoch 3/200
47/47 [==============================] - 0s 3ms/step - loss: 113.0822 - mean_absolute_percentage_error: 6.5496 - val_loss: 143.9727 - val_mean_absolute_percentage_error: 8.4449
Epoch 4/200
47/47 [==============================] - 0s 3ms/step - loss: 85.8907 - mean_absolute_percentage_error: 5.2601 - val_loss: 87.0984 - val_mean_absolute_percentage_error: 6.1702
Epoch 5/200
47/47 [==============================] - 0s 3ms/step - loss: 110.4080 - mean_absolute_percentage_error: 5.9489 - val_loss: 11.3613 - val_mean_absolute_percentage_error: 1.7497
Epoch 6/200
47/47 [==============================] - 0s 3

In [63]:
y_pred_ann = ann_relu.predict(x_test)
#print(y_pred_ann)

In [64]:
mse_ann = mean_squared_error(y_test, y_pred_ann)
print(f'The MSE for the ANN algorithm is: {mse_ann}')

The MSE for the ANN algorithm is: 9.993550807359448


In [65]:
rmse_ann = mean_squared_error(y_test, y_pred_ann, squared=False)
print(f'The RMSE for the ANN algorithm is: {rmse_ann}')

The RMSE for the ANN algorithm is: 3.161257788817522


In [66]:
mae_ann = mean_absolute_error(y_test, y_pred_ann)
print(f'The MAE for the ANN algorithm is: {mae_ann}')

The MAE for the ANN algorithm is: 1.8866583045208645


In [67]:
def mean_absolute_percentage_error(y_test, y_pred_ann): 
    y_test, y_pred_ann = np.array(y_test), np.array(y_pred_ann)
    return np.mean(np.abs((y_test - y_pred_ann) / y_test)) * 100
    return mean_absolute_percentage_error

In [68]:
mae_ann = mean_absolute_error(y_test, y_pred_ann)
print(mae_ann/y_test)

[0.09720032 0.00865201 0.04438152 0.01101634 0.01182117 0.00790091
 0.09675171 0.0250752  0.01510777 0.01334082 0.0104669  0.02424076
 0.00932743 0.01023633 0.06918439 0.01328165 0.03734478 0.0104172
 0.00855084 0.01118152 0.00747103 0.0170476  0.06512456 0.02000698
 0.00937051 0.04505991 0.01059325 0.01126363 0.0158636  0.010792
 0.00919783 0.01118285 0.00712888 0.00707355 0.00717061 0.01639432
 0.04069582 0.02939178 0.01351184 0.01367838 0.0152285  0.0329893
 0.01567252 0.01582236 0.07694365 0.02665525 0.00786534 0.07312629
 0.0209396  0.02146857 0.02049602 0.04900411 0.01484389 0.01372914
 0.00681178 0.01754868 0.01432107 0.0100536  0.01531627 0.01998367
 0.01518804 0.02435655 0.01049835 0.01327791 0.00722028 0.00703741
 0.00885506 0.01476605 0.01392985 0.07231347 0.00939243 0.03185309
 0.01214144 0.00948928 0.00977543 0.01406065 0.01161521 0.01179383
 0.03244468 0.01278484 0.00976582 0.02405531 0.0070887  0.02465898
 0.0151966  0.01228373 0.04063447 0.02702949 0.01731038 0.0995073


In [69]:
keras.backend.clear_session()